In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 62.4 MB/s eta 0:00:00


MessageError: Error: credential propagation was unsuccessful

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/Kaggle_credentials/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting

In [ ]:
! unzip /content/walmart-recruiting-store-sales-forecasting.zip
! unzip /content/train.csv.zip
! unzip /content/test.csv.zip
! unzip /content/features.csv.zip
! unzip /content/sampleSubmission.csv.zip

In [ ]:
!pip install wandb -qU

In [ ]:
import wandb
import random
import math

In [ ]:
wandb.login()

In [ ]:
# Initialize wandb for pipeline development
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ARIMA and time series imports
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima

# Sklearn imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import wandb
from datetime import datetime

# DATA EXPLORATION

In [ ]:
# Change project name in all wandb.init() calls:
wandb.init(project="walmart-sales-forecasting", name="Data_Exploration", tags=["exploration"])

print("=== WALMART SALES FORECASTING - DATA EXPLORATION ===\n")

# Load all datasets
print("Loading datasets...")
train = pd.read_csv("/content/train.csv")
features = pd.read_csv("/content/features.csv")
stores = pd.read_csv("/content/stores.csv")
test = pd.read_csv("/content/test.csv")
sample_submission = pd.read_csv("/content/sampleSubmission.csv")

print("Dataset shapes:")
print(f"Train: {train.shape}")
print(f"Features: {features.shape}")
print(f"Stores: {stores.shape}")
print(f"Test: {test.shape}")
print(f"Sample Submission: {sample_submission.shape}")

# Log basic dataset info
wandb.log({
    "train_rows": train.shape[0],
    "train_cols": train.shape[1],
    "features_rows": features.shape[0],
    "features_cols": features.shape[1],
    "stores_count": stores.shape[0],
    "test_rows": test.shape[0]
})

In [ ]:
# Explore individual datasets
print("\n=== TRAIN DATASET ===")
print("Columns:", train.columns.tolist())
print("\nFirst 5 rows:")
print(train.head())
print("\nBasic statistics:")
print(train.describe())
print(f"\nMissing values:\n{train.isnull().sum()}")

print("\n=== FEATURES DATASET ===")
print("Columns:", features.columns.tolist())
print(f"\nMissing values:\n{features.isnull().sum()}")
print("\nFeatures data sample:")
print(features.head())

print("\n=== STORES DATASET ===")
print("Columns:", stores.columns.tolist())
print(f"\nStore types: {stores['Type'].value_counts().to_dict()}")
print(f"Store sizes range: {stores['Size'].min()} - {stores['Size'].max()}")
print(stores.head())

In [ ]:
# Merge datasets for exploration
print("\n=== MERGING DATASETS ===")
data = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
data = data.merge(stores, on='Store', how='left')

print(f"Merged dataset shape: {data.shape}")
print(f"Missing values after merge:\n{data.isnull().sum()}")

# Convert date to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Basic time series analysis
print(f"\nDate range: {data['Date'].min()} to {data['Date'].max()}")
print(f"Unique stores: {data['Store'].nunique()}")
print(f"Unique departments: {data['Dept'].nunique()}")
print(f"Total weeks: {data['Date'].nunique()}")

# Log exploration metrics
wandb.log({
    "merged_shape_rows": data.shape[0],
    "merged_shape_cols": data.shape[1],
    "unique_stores": data['Store'].nunique(),
    "unique_departments": data['Dept'].nunique(),
    "unique_weeks": data['Date'].nunique(),
    "missing_values_total": data.isnull().sum().sum()
})

# SOME VISUALS

In [ ]:
# Visualize sales distribution and patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sales distribution
axes[0, 0].hist(data['Weekly_Sales'], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribution of Weekly Sales')
axes[0, 0].set_xlabel('Weekly Sales')
axes[0, 0].set_ylabel('Frequency')

# Sales by store type
store_sales = data.groupby('Type')['Weekly_Sales'].mean().reset_index()
axes[0, 1].bar(store_sales['Type'], store_sales['Weekly_Sales'], color=['coral', 'lightgreen', 'gold'])
axes[0, 1].set_title('Average Weekly Sales by Store Type')
axes[0, 1].set_xlabel('Store Type')
axes[0, 1].set_ylabel('Average Weekly Sales')

# Sales over time (monthly average)
data['Month'] = data['Date'].dt.month
monthly_sales = data.groupby('Month')['Weekly_Sales'].mean().reset_index()
axes[1, 0].plot(monthly_sales['Month'], monthly_sales['Weekly_Sales'], marker='o', color='purple')
axes[1, 0].set_title('Average Sales by Month')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average Weekly Sales')
axes[1, 0].set_xticks(range(1, 13))

# Holiday vs Non-holiday sales
holiday_sales = data.groupby('IsHoliday')['Weekly_Sales'].mean()
axes[1, 1].bar(['Non-Holiday', 'Holiday'], holiday_sales.values, color=['lightcoral', 'lightsalmon'])
axes[1, 1].set_title('Average Sales: Holiday vs Non-Holiday')
axes[1, 1].set_ylabel('Average Weekly Sales')

plt.tight_layout()
plt.savefig('data_exploration.png', dpi=300, bbox_inches='tight')
wandb.log({"data_exploration": wandb.Image('data_exploration.png')})
plt.show()

# Correlation analysis

In [ ]:
# Correlation analysis
print("\n=== CORRELATION ANALYSIS ===")
numeric_cols = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']
correlation_matrix = data[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix of Numeric Features')
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
wandb.log({"correlation_matrix": wandb.Image('correlation_matrix.png')})
plt.show()

print("Key correlations with Weekly_Sales:")
sales_corr = correlation_matrix['Weekly_Sales'].sort_values(key=abs, ascending=False)
for feature, corr in sales_corr.items():
    if feature != 'Weekly_Sales':
        print(f"{feature}: {corr:.3f}")

wandb.finish()

# Pipeline Definition

In [ ]:
# Initialize wandb for pipeline development
wandb.init(project="walmart-sales-forecasting", name="ARIMA_Pipeline_Development", tags=["pipeline", "arima"])

print("=== DEVELOPING PREPROCESSING PIPELINE ===\n")

class WalmartARIMAPreprocessor(BaseEstimator, TransformerMixin):
    """
    Custom preprocessor for Walmart sales data for ARIMA modeling:
    - Time series structure preparation
    - Stationarity checks
    - Seasonal decomposition
    - Exogenous variable preparation
    """

    def __init__(self, freq='W', include_exogenous=True, aggregate_level='store_dept'):
        self.freq = freq  # Weekly frequency
        self.include_exogenous = include_exogenous
        self.aggregate_level = aggregate_level  # 'store_dept', 'store', or 'total'
        self.exog_columns = []
        self.time_series_data = {}
        self.fitted = False

    def fit(self, X, y=None):
        """Fit the preprocessor on training data"""
        print("Fitting preprocessor...")

        # Combine X and y for time series preparation
        data = X.copy()
        if y is not None:
            data['Weekly_Sales'] = y

        # Convert Date to datetime
        data['Date'] = pd.to_datetime(data['Date'])

        # Prepare exogenous variables
        if self.include_exogenous:
            self.exog_columns = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'IsHoliday']
            # Add Type encoding
            if 'Type' in data.columns:
                data['Type_A'] = (data['Type'] == 'A').astype(int)
                data['Type_B'] = (data['Type'] == 'B').astype(int)
                self.exog_columns.extend(['Type_A', 'Type_B'])

        self.fitted = True
        print("Preprocessor fitted successfully!")
        return self

    def transform(self, X):
        """Transform the data for ARIMA modeling"""
        if not self.fitted:
            raise ValueError("Preprocessor must be fitted before transform!")

        data = X.copy()
        data['Date'] = pd.to_datetime(data['Date'])

        # Add Type encoding if needed
        if 'Type' in data.columns and self.include_exogenous:
            data['Type_A'] = (data['Type'] == 'A').astype(int)
            data['Type_B'] = (data['Type'] == 'B').astype(int)

        return data

    def prepare_time_series(self, data, target_col='Weekly_Sales'):
        """Prepare individual time series for ARIMA modeling"""
        time_series_dict = {}

        if self.aggregate_level == 'store_dept':
            # Group by Store and Dept - most granular
            for name, group in data.groupby(['Store', 'Dept']):
                ts_data = self._prepare_single_series(group, target_col, name)
                if ts_data is not None:
                    time_series_dict[name] = ts_data

        elif self.aggregate_level == 'store':
            # Group by Store only
            for name, group in data.groupby(['Store']):
                # Aggregate by store
                agg_dict = {'Weekly_Sales': 'sum', 'Date': 'first'}
                for col in self.exog_columns:
                    if col in group.columns:
                        agg_dict[col] = 'mean'

                store_data = group.groupby('Date').agg(agg_dict).reset_index()
                ts_data = self._prepare_single_series(store_data, target_col, name)
                if ts_data is not None:
                    time_series_dict[name] = ts_data
        else:
            # Total aggregation
            agg_dict = {'Weekly_Sales': 'sum'}
            for col in self.exog_columns:
                if col in data.columns:
                    agg_dict[col] = 'mean'

            total_data = data.groupby('Date').agg(agg_dict).reset_index()
            ts_data = self._prepare_single_series(total_data, target_col, 'total')
            if ts_data is not None:
                time_series_dict['total'] = ts_data

        return time_series_dict

    def _prepare_single_series(self, group, target_col, name):
        """Prepare a single time series"""
        try:
            # Sort by date and set as index
            ts_data = group.sort_values('Date').set_index('Date')

            # Create complete date range and reindex
            date_range = pd.date_range(start=ts_data.index.min(),
                                     end=ts_data.index.max(),
                                     freq=self.freq)
            ts_data = ts_data.reindex(date_range)

            # Handle exogenous variables
            if self.include_exogenous:
                for col in self.exog_columns:
                    if col in ts_data.columns:
                        ts_data[col] = ts_data[col].fillna(method='ffill').fillna(method='bfill')

            # Handle missing target values
            if target_col in ts_data.columns:
                ts_data[target_col] = ts_data[target_col].fillna(ts_data[target_col].median())

                # Remove series with too few observations
                if ts_data[target_col].count() < 20:
                    return None

            return ts_data

        except Exception as e:
            print(f"Warning: Failed to prepare series {name}: {str(e)}")
            return None

print("ARIMA Preprocessor class defined successfully!")
# Test the preprocessor
print("Testing preprocessor with sample data...")

# Use a subset of data for testing
sample_data = data.head(1000).copy()
X_sample = sample_data.drop('Weekly_Sales', axis=1)
y_sample = sample_data['Weekly_Sales']

# Initialize and fit preprocessor
arima_preprocessor = WalmartARIMAPreprocessor(aggregate_level='store')
arima_preprocessor.fit(X_sample, y_sample)

# Transform data
X_transformed = arima_preprocessor.transform(X_sample.assign(Weekly_Sales=y_sample))

# Prepare time series
ts_data_dict = arima_preprocessor.prepare_time_series(X_transformed)

print(f"Original features: {X_sample.shape[1]}")
print(f"Time series created: {len(ts_data_dict)}")
if ts_data_dict:
    sample_ts = list(ts_data_dict.values())[0]
    print(f"Sample time series shape: {sample_ts.shape}")
    feature_names = list(sample_ts.columns)
    print(f"New feature names: {feature_names}")

# Log preprocessing results
wandb.log({
    "original_features": X_sample.shape[1],
    "transformed_features": len(ts_data_dict),
    "feature_engineering_ratio": len(ts_data_dict) / X_sample.shape[1] if X_sample.shape[1] > 0 else 0
})

print("\nPreprocessor created successfully!")
wandb.finish()

# DATA PREPROCESSING AND INITIAL TRAINING

In [ ]:
# Initialize wandb for preprocessing and training
wandb.init(project="walmart-sales-forecasting", name="ARIMA_Preprocessing_Training", tags=["preprocessing", "training", "arima"])

print("=== DATA PREPROCESSING AND INITIAL TRAINING ===\n")

# Load and prepare full dataset
print("Loading and preparing full dataset...")
data = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
data = data.merge(stores, on='Store', how='left')

print(f"Full dataset shape: {data.shape}")

# Remove rows with missing target
initial_rows = data.shape[0]
data = data.dropna(subset=['Weekly_Sales'])
final_rows = data.shape[0]

print(f"Removed {initial_rows - final_rows} rows with missing target")
print(f"Final dataset shape: {data.shape}")

# Sort by store, department, and date for time series
data = data.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

# Prepare features and target
X = data.drop('Weekly_Sales', axis=1)
y = data['Weekly_Sales']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target statistics:")
print(f"  Mean: {y.mean():.2f}")
print(f"  Std: {y.std():.2f}")
print(f"  Min: {y.min():.2f}")
print(f"  Max: {y.max():.2f}")

# Log data statistics
wandb.log({
    "dataset_rows_final": final_rows,
    "features_count": X.shape[1],
    "target_mean": y.mean(),
    "target_std": y.std(),
    "target_min": y.min(),
    "target_max": y.max(),
    "rows_removed": initial_rows - final_rows
})

# Preprocessing


In [ ]:
# Create and fit preprocessor
print("\nFitting preprocessor on full dataset...")
preprocessor = WalmartDataPreprocessor(
    create_lag_features=True,
    lag_periods=[1, 2, 4, 8],
    rolling_windows=[4, 8, 12],
    create_interactions=True
)

# Fit preprocessor
preprocessor.fit(X)

# Transform features
print("Transforming features...")
X_processed = preprocessor.transform(X)

print(f"Processed features shape: {X_processed.shape}")
print(f"Feature engineering created {X_processed.shape[1] - X.shape[1]} new features")

# Check for any remaining missing values
missing_counts = X_processed.isnull().sum()
if missing_counts.sum() > 0:
    print(f"Warning: Still have missing values:")
    print(missing_counts[missing_counts > 0])
else:
    print("✓ No missing values in processed data")

# Log preprocessing results
wandb.log({
    "processed_features_count": X_processed.shape[1],
    "new_features_created": X_processed.shape[1] - X.shape[1],
    "remaining_missing_values": missing_counts.sum()
})

#  INITIAL MODEL TRAINING

In [ ]:
# Create and fit preprocessor
print("\nFitting preprocessor on full dataset...")
arima_preprocessor = WalmartARIMAPreprocessor(
    freq='W',
    include_exogenous=True,
    aggregate_level='store'  # Aggregate by store for manageable number of models
)

# Fit preprocessor
arima_preprocessor.fit(X, y)

# Transform features
print("Transforming features...")
data_transformed = arima_preprocessor.transform(X.assign(Weekly_Sales=y))

# Prepare time series
ts_data_dict = arima_preprocessor.prepare_time_series(data_transformed)

print(f"Time series created: {len(ts_data_dict)}")
print(f"Time series preparation created {len(ts_data_dict)} individual models needed")

# Check for any remaining missing values in sample series
if ts_data_dict:
    sample_ts = list(ts_data_dict.values())[0]
    missing_counts = sample_ts.isnull().sum()
    if missing_counts.sum() > 0:
        print(f"Warning: Still have missing values in sample series:")
        print(missing_counts[missing_counts > 0])
    else:
        print("✓ No missing values in processed time series")

# Log preprocessing results
wandb.log({
    "processed_features_count": len(ts_data_dict),
    "new_features_created": len(ts_data_dict) - X.shape[1],
    "remaining_missing_values": missing_counts.sum() if 'missing_counts' in locals() else 0
})

print("\n=== INITIAL ARIMA MODEL TRAINING ===")

# Initialize ARIMA ensemble for initial training
initial_arima = ARIMAEnsemble(
    max_p=2,  # Reduced for initial training
    max_d=1,
    max_q=2,
    seasonal=True,
    m=52
)

# Train on subset for initial assessment
sample_series = dict(list(ts_data_dict.items())[:5])  # First 5 series
print(f"Training initial ARIMA models on {len(sample_series)} time series...")

exog_cols = arima_preprocessor.exog_columns
successful_fits, failed_fits = initial_arima.fit(sample_series, exog_cols=exog_cols)

# Get initial performance estimates
if successful_fits > 0:
    fitted_values_dict = initial_arima.get_fitted_values()

    initial_maes = []
    initial_rmses = []

    for name, fitted_values in fitted_values_dict.items():
        if name in sample_series:
            y_actual = sample_series[name]['Weekly_Sales'].dropna()
            common_index = y_actual.index.intersection(fitted_values.index)

            if len(common_index) > 0:
                y_aligned = y_actual.loc[common_index]
                fitted_aligned = fitted_values.loc[common_index]

                mae = mean_absolute_error(y_aligned, fitted_aligned)
                rmse = np.sqrt(mean_squared_error(y_aligned, fitted_aligned))

                initial_maes.append(mae)
                initial_rmses.append(rmse)

    if initial_maes:
        avg_mae = np.mean(initial_maes)
        avg_rmse = np.mean(initial_rmses)

        print(f"\nInitial Model Results (sample of {len(initial_maes)} models):")
        print(f"Average MAE: {avg_mae:.2f}")
        print(f"Average RMSE: {avg_rmse:.2f}")
        print(f"MAE Range: {min(initial_maes):.2f} - {max(initial_maes):.2f}")

        # Log initial results
        wandb.log({
            "initial_sample_models": len(initial_maes),
            "initial_avg_mae": avg_mae,
            "initial_avg_rmse": avg_rmse,
            "initial_min_mae": min(initial_maes),
            "initial_max_mae": max(initial_maes),
            "initial_successful_fits": successful_fits,
            "initial_failed_fits": failed_fits
        })

print(f"\nSuccessfully fitted {successful_fits} initial ARIMA models!")
wandb.finish()

In [ ]:
# Initialize wandb for cross-validation and tuning
wandb.init(project="walmart-sales-forecasting", name="ARIMA_CrossValidation_Tuning", tags=["cv", "tuning", "arima"])

print("=== CROSS-VALIDATION AND HYPERPARAMETER TUNING ===\n")

# ARIMA parameter grid for tuning
param_grid = {
    'max_p': [2, 3],
    'max_d': [1, 2],
    'max_q': [2, 3],
    'seasonal': [True],  # Always use seasonal
    'm': [52]  # Weekly seasonality
}

print(f"ARIMA hyperparameter search space:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

total_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"\nTotal combinations to test: {total_combinations}")

def evaluate_arima_params(params, ts_sample, cv_splits=3, test_size=8):
    """Evaluate ARIMA parameters using time series cross-validation"""
    all_cv_scores = []

    for name, ts_data in ts_sample.items():
        try:
            y = ts_data['Weekly_Sales'].dropna()

            if len(y) < test_size * (cv_splits + 1) + 20:
                continue

            cv_scores = []

            for i in range(cv_splits):
                test_end = len(y) - i * test_size
                test_start = test_end - test_size
                train_end = test_start

                if train_end < 20:
                    break

                y_train = y.iloc[:train_end]
                y_test = y.iloc[test_start:test_end]

                try:
                    model = auto_arima(
                        y_train,
                        start_p=0, start_q=0,
                        max_p=params['max_p'],
                        max_d=params['max_d'],
                        max_q=params['max_q'],
                        seasonal=params['seasonal'],
                        m=params['m'],
                        stepwise=True,
                        suppress_warnings=True,
                        error_action='ignore'
                    )

                    predictions = model.predict(n_periods=len(y_test))
                    mae = mean_absolute_error(y_test, predictions)
                    cv_scores.append(mae)

                except:
                    continue

            if cv_scores:
                all_cv_scores.extend(cv_scores)

        except:
            continue

    return all_cv_scores

# Hyperparameter tuning
print("\nStarting hyperparameter tuning...")
best_score = float('inf')
best_params = None
all_results = []

# Use subset of time series for tuning
tuning_sample = dict(list(ts_data_dict.items())[:3])  # First 3 series
print(f"Using {len(tuning_sample)} time series for hyperparameter tuning")

iteration = 0
for max_p in param_grid['max_p']:
    for max_d in param_grid['max_d']:
        for max_q in param_grid['max_q']:
            for seasonal in param_grid['seasonal']:
                for m in param_grid['m']:
                    iteration += 1

                    params = {
                        'max_p': max_p,
                        'max_d': max_d,
                        'max_q': max_q,
                        'seasonal': seasonal,
                        'm': m
                    }

                    print(f"\n[{iteration}/{total_combinations}] Testing parameters:")
                    print(f"  max_p={max_p}, max_d={max_d}, max_q={max_q}, seasonal={seasonal}, m={m}")

                    # Evaluate with cross-validation
                    cv_scores = evaluate_arima_params(params, tuning_sample)

                    if cv_scores:
                        avg_score = np.mean(cv_scores)
                        std_score = np.std(cv_scores)

                        print(f"  CV Score: {avg_score:.3f} (+/- {std_score:.3f}) from {len(cv_scores)} folds")

                        # Log to wandb
                        wandb.log({
                            **params,
                            'cv_mae_mean': avg_score,
                            'cv_mae_std': std_score,
                            'cv_mae_min': min(cv_scores),
                            'cv_mae_max': max(cv_scores),
                            'cv_folds': len(cv_scores),
                            'iteration': iteration
                        })

                        # Store results
                        result = {
                            'params': params.copy(),
                            'cv_mean': avg_score,
                            'cv_std': std_score,
                            'cv_scores': cv_scores
                        }
                        all_results.append(result)

                        # Update best parameters
                        if avg_score < best_score:
                            best_score = avg_score
                            best_params = params.copy()
                            print(f"  *** NEW BEST SCORE: {best_score:.3f} ***")
                    else:
                        print(f"  No valid CV scores obtained")

print(f"\n=== HYPERPARAMETER TUNING COMPLETED ===")
if best_params:
    print(f"Best CV Score: {best_score:.3f}")
    print(f"Best Parameters: {best_params}")

    # Log best results
    wandb.log({
        'best_cv_mae': best_score,
        'best_params': best_params,
        'total_combinations_tested': total_combinations
    })
else:
    print("No valid parameter combinations found")
    best_params = {
        'max_p': 3,
        'max_d': 1,
        'max_q': 3,
        'seasonal': True,
        'm': 52
    }
    print(f"Using default parameters: {best_params}")

# Analysis of results
if all_results:
    results_df = pd.DataFrame([
        {**r['params'], 'cv_mean': r['cv_mean'], 'cv_std': r['cv_std']}
        for r in all_results
    ])

    print(f"\nTop parameter combinations:")
    top_results = results_df.nsmallest(min(5, len(results_df)), 'cv_mean')[['max_p', 'max_d', 'max_q', 'cv_mean', 'cv_std']]
    print(top_results)

wandb.finish()

# FINAL MODEL TRAINING AND ANALYSIS

In [ ]:
# Initialize wandb for final training
wandb.init(project="walmart-sales-forecasting", name="ARIMA_Final_Training", tags=["final", "training", "arima", "production"])

print("=== FINAL MODEL TRAINING AND ANALYSIS ===\n")

# Load best parameters (use best found or defaults if none)
final_params = best_params if 'best_params' in locals() and best_params else {
    'max_p': 3,
    'max_d': 1,
    'max_q': 3,
    'seasonal': True,
    'm': 52
}

print("Final model parameters:")
for param, value in final_params.items():
    print(f"  {param}: {value}")

# Log final parameters
wandb.log(final_params)

# Create final ARIMA ensemble
print("\nCreating final ARIMA ensemble...")

final_arima_ensemble = ARIMAEnsemble(
    max_p=final_params['max_p'],
    max_d=final_params['max_d'],
    max_q=final_params['max_q'],
    seasonal=final_params['seasonal'],
    m=final_params['m']
)

print("ARIMA ensemble created successfully!")

# Train final model on full dataset
print("\nTraining final ARIMA ensemble on full dataset...")

print(f"Training on {len(ts_data_dict)} time series")

# Fit ensemble
exog_cols = arima_preprocessor.exog_columns
successful_fits, failed_fits = final_arima_ensemble.fit(ts_data_dict, exog_cols=exog_cols)

print("Final ARIMA ensemble trained successfully!")
print(f"Successfully fitted: {successful_fits} models")
print(f"Failed fits: {failed_fits} models")

# Make predictions for evaluation
print("Making predictions on training data...")

# Get fitted values from all models
fitted_values_dict = final_arima_ensemble.get_fitted_values()

all_predictions_final = []
all_actuals_final = []
model_performance_final = {}

for name, fitted_values in fitted_values_dict.items():
    if name in ts_data_dict:
        try:
            ts_data = ts_data_dict[name]
            y_actual = ts_data['Weekly_Sales'].dropna()

            # Align fitted values with actual values
            common_index = y_actual.index.intersection(fitted_values.index)

            if len(common_index) > 0:
                y_aligned = y_actual.loc[common_index]
                fitted_aligned = fitted_values.loc[common_index]

                # Calculate metrics for this model
                mae = mean_absolute_error(y_aligned, fitted_aligned)
                rmse = np.sqrt(mean_squared_error(y_aligned, fitted_aligned))
                r2 = r2_score(y_aligned, fitted_aligned)

                model_performance_final[name] = {
                    'mae': mae,
                    'rmse': rmse,
                    'r2': r2,
                    'observations': len(y_aligned)
                }

                # Collect for overall metrics
                all_predictions_final.extend(fitted_aligned.values)
                all_actuals_final.extend(y_aligned.values)

        except Exception as e:
            print(f"Evaluation failed for {name}: {str(e)}")
            continue

# Calculate comprehensive metrics
if all_predictions_final and all_actuals_final:
    train_mae = mean_absolute_error(all_actuals_final, all_predictions_final)
    train_rmse = np.sqrt(mean_squared_error(all_actuals_final, all_predictions_final))
    train_r2 = r2_score(all_actuals_final, all_actuals_final)

    # Fix MAPE calculation (handle zero values)
    def safe_mape(y_true, y_pred):
        """Calculate MAPE while handling zero values"""
        mask = np.array(y_true) != 0
        if mask.sum() == 0:
            return float('inf')
        return np.mean(np.abs((np.array(y_true)[mask] - np.array(y_pred)[mask]) / np.array(y_true)[mask])) * 100

    train_mape_fixed = safe_mape(all_actuals_final, all_predictions_final)

    print(f"\n=== FINAL MODEL PERFORMANCE ===")
    print(f"Training MAE: {train_mae:.2f}")
    print(f"Training RMSE: {train_rmse:.2f}")
    print(f"Training MAPE: {train_mape_fixed:.2f}%")
    print(f"Training R²: {train_r2:.4f}")

    # Log final metrics
    wandb.log({
        'final_train_mae': train_mae,
        'final_train_rmse': train_rmse,
        'final_train_mape': train_mape_fixed if not np.isinf(train_mape_fixed) else 0.0,
        'final_train_r2': train_r2,
        'final_samples_count': len(all_actuals_final)
    })

    # Individual model statistics
    individual_maes = [perf['mae'] for perf in model_performance_final.values()]

    print(f"\nIndividual Model Performance:")
    print(f"  Best MAE: {min(individual_maes):.2f}")
    print(f"  Worst MAE: {max(individual_maes):.2f}")
    print(f"  Median MAE: {np.median(individual_maes):.2f}")

# Get feature names equivalent
feature_names = arima_preprocessor.exog_columns

print("\n=== SAVING FINAL MODEL ===")

# Save ARIMA ensemble filename
pipeline_filename = f"arima_pipeline_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"

# Create comprehensive save object
arima_save_object = {
    'ensemble': final_arima_ensemble,
    'preprocessor': arima_preprocessor,
    'performance': model_performance_final,
    'overall_metrics': {
        'mae': train_mae if 'train_mae' in locals() else 0,
        'rmse': train_rmse if 'train_rmse' in locals() else 0,
        'mape': train_mape_fixed if 'train_mape_fixed' in locals() and not np.isinf(train_mape_fixed) else 0.0,
        'r2': train_r2 if 'train_r2' in locals() else 0
    },
    'metadata': {
        'created_date': datetime.now().isoformat(),
        'successful_models': successful_fits,
        'total_series': len(ts_data_dict),
        'exogenous_variables': feature_names
    }
}

# Save with cloudpickle
try:
    import cloudpickle
    with open(pipeline_filename, 'wb') as f:
        cloudpickle.dump(arima_save_object, f)
    print(f"Pipeline saved as: {pipeline_filename}")
except ImportError:
    import pickle
    with open(pipeline_filename, 'wb') as f:
        pickle.dump(arima_save_object, f)
    print(f"Pipeline saved as: {pipeline_filename}")

# Try to upload to wandb with error handling
try:
    model_artifact = wandb.Artifact(
        name="ARIMA_pipeline",
        type="model",
        description="Final ARIMA pipeline for Walmart sales forecasting",
        metadata={
            "train_mae": float(train_mae) if 'train_mae' in locals() else 0.0,
            "train_rmse": float(train_rmse) if 'train_rmse' in locals() else 0.0,
            "train_mape": float(train_mape_fixed) if 'train_mape_fixed' in locals() and not np.isinf(train_mape_fixed) else 0.0,
            "train_r2": float(train_r2) if 'train_r2' in locals() else 0.0,
            "features_count": len(feature_names),
            "training_samples": len(all_actuals_final) if 'all_actuals_final' in locals() else 0,
            "model_type": "ARIMA"
        }
    )

    model_artifact.add_file(pipeline_filename)
    wandb.log_artifact(model_artifact)
    print("✓ Model artifact logged to wandb successfully!")

except Exception as e:
    print(f"⚠️ Error uploading to wandb: {e}")
    print("Model saved locally - you can manually upload later")

    # Log just the metrics without artifact
    wandb.log({
        'final_train_mae': train_mae if 'train_mae' in locals() else 0,
        'final_train_rmse': train_rmse if 'train_rmse' in locals() else 0,
        'final_train_mape': train_mape_fixed if 'train_mape_fixed' in locals() and not np.isinf(train_mape_fixed) else 0.0,
        'final_train_r2': train_r2 if 'train_r2' in locals() else 0,
        'model_saved_locally': pipeline_filename
    })

# Final summary
print(f"\n" + "="*60)
print("FINAL MODEL SUMMARY")
print("="*60)
print(f"Model Type: ARIMA Ensemble")
print(f"Training Samples: {len(all_actuals_final) if 'all_actuals_final' in locals() else 0:,}")
print(f"Individual Models: {successful_fits}")
print(f"Features: {len(feature_names)}")
if 'train_mae' in locals():
    print(f"Training MAE: {train_mae:.2f}")
    print(f"Training RMSE: {train_rmse:.2f}")
    print(f"Training MAPE: {train_mape_fixed:.2f}%")
    print(f"Training R²: {train_r2:.4f}")
print(f"Pipeline saved as: {pipeline_filename}")
print("="*60)

wandb.finish()

# Final pipeline

In [ ]:
# Create final pipeline equivalent
print("\nCreating final ARIMA pipeline...")

final_preprocessor = WalmartARIMAPreprocessor(
    freq='W',
    include_exogenous=True,
    aggregate_level='store'
)

final_model = ARIMAEnsemble(
    max_p=final_params['max_p'],
    max_d=final_params['max_d'],
    max_q=final_params['max_q'],
    seasonal=final_params['seasonal'],
    m=final_params['m']
)

print("ARIMA Pipeline created successfully!")

# Final model training

In [ ]:
# Train final model on full dataset
print("\nTraining final model on full dataset...")

# Prepare final dataset
X_final = data.drop('Weekly_Sales', axis=1)
y_final = data['Weekly_Sales']

print(f"Training on {len(X_final)} samples with {X_final.shape[1]} features")

# Fit preprocessor
final_preprocessor.fit(X_final, y_final)

# Transform data
data_transformed = final_preprocessor.transform(X_final.assign(Weekly_Sales=y_final))

# Prepare time series
ts_data_dict = final_preprocessor.prepare_time_series(data_transformed)

# Fit ensemble
exog_cols = final_preprocessor.exog_columns
successful_fits, failed_fits = final_model.fit(ts_data_dict, exog_cols=exog_cols)

print("Final ARIMA ensemble trained successfully!")
print(f"Successfully fitted: {successful_fits} models")
print(f"Failed fits: {failed_fits} models")

# Make predictions for evaluation
print("Making predictions on training data...")

# Get fitted values from all models
fitted_values_dict = final_model.get_fitted_values()

all_predictions = []
all_actuals = []

for name, fitted_values in fitted_values_dict.items():
    if name in ts_data_dict:
        try:
            ts_data = ts_data_dict[name]
            y_actual = ts_data['Weekly_Sales'].dropna()
            common_index = y_actual.index.intersection(fitted_values.index)

            if len(common_index) > 0:
                y_aligned = y_actual.loc[common_index]
                fitted_aligned = fitted_values.loc[common_index]

                all_predictions.extend(fitted_aligned.values)
                all_actuals.extend(y_aligned.values)
        except Exception as e:
            print(f"Evaluation failed for {name}: {str(e)}")
            continue

y_pred_final = np.array(all_predictions)
y_final_aligned = np.array(all_actuals)

# Calculate comprehensive metrics
train_mae = mean_absolute_error(y_final_aligned, y_pred_final)
train_rmse = np.sqrt(mean_squared_error(y_final_aligned, y_pred_final))
train_mape = np.mean(np.abs((y_final_aligned - y_pred_final) / y_final_aligned)) * 100

# Calculate R²
from sklearn.metrics import r2_score
train_r2 = r2_score(y_final_aligned, y_pred_final)

print(f"\n=== FINAL MODEL PERFORMANCE ===")
print(f"Training MAE: {train_mae:.2f}")
print(f"Training RMSE: {train_rmse:.2f}")
print(f"Training MAPE: {train_mape:.2f}%")
print(f"Training R²: {train_r2:.4f}")

# Log final metrics
wandb.log({
    'final_train_mae': train_mae,
    'final_train_rmse': train_rmse,
    'final_train_mape': train_mape,
    'final_train_r2': train_r2,
    'final_samples_count': len(y_final_aligned)
})

# Analysis

In [ ]:
# ARIMA Model Analysis (equivalent to feature importance)
print("\n=== ARIMA MODEL ANALYSIS ===")

# Analyze exogenous variable usage across models
exog_usage = {}
model_orders = {}
seasonal_orders = {}

for name, params in final_model.model_params.items():
    # Track exogenous variable usage
    for exog_var in params.get('exog_used', []):
        exog_usage[exog_var] = exog_usage.get(exog_var, 0) + 1

    # Track model orders
    order_str = str(params['order'])
    model_orders[order_str] = model_orders.get(order_str, 0) + 1

    # Track seasonal orders
    if params.get('seasonal_order'):
        seasonal_str = str(params['seasonal_order'])
        seasonal_orders[seasonal_str] = seasonal_orders.get(seasonal_str, 0) + 1

# Create exogenous variable importance based on usage
total_models = len(final_model.models)
feature_names = list(exog_usage.keys()) if exog_usage else final_preprocessor.exog_columns

if exog_usage:
    importance_df = pd.DataFrame([
        {
            'Feature': var,
            'Models_Used': count,
            'Usage_Pct': (count / total_models) * 100,
            'Importance': count / total_models  # Normalized importance
        }
        for var, count in exog_usage.items()
    ]).sort_values('Usage_Pct', ascending=False)

    print(f"Exogenous Variable Usage Across {total_models} Models:")
    for i, (_, row) in enumerate(importance_df.iterrows()):
        print(f"{i+1:2d}. {row['Feature']:<25} | {row['Models_Used']}/{total_models} models ({row['Usage_Pct']:.1f}%)")
else:
    print("No exogenous variables were used in the models")
    importance_df = pd.DataFrame(columns=['Feature', 'Models_Used', 'Usage_Pct', 'Importance'])

print(f"\nMost Common ARIMA Orders:")
for i, (order, count) in enumerate(sorted(model_orders.items(), key=lambda x: x[1], reverse=True)[:10]):
    print(f"{i+1:2d}. {order:<15} | {count} models ({count/total_models*100:.1f}%)")

if seasonal_orders:
    print(f"\nMost Common Seasonal Orders:")
    for i, (order, count) in enumerate(sorted(seasonal_orders.items(), key=lambda x: x[1], reverse=True)[:5]):
        print(f"{i+1:2d}. {order:<20} | {count} models ({count/total_models*100:.1f}%)")

# Create visualization
import matplotlib.pyplot as plt

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))

# Exogenous variable usage
if not importance_df.empty:
    top_vars = importance_df.head(10)
    ax1.barh(range(len(top_vars)), top_vars['Usage_Pct'])
    ax1.set_yticks(range(len(top_vars)))
    ax1.set_yticklabels(top_vars['Feature'])
    ax1.set_xlabel('Usage Percentage (%)')
    ax1.set_title('Exogenous Variable Usage Across Models')
    ax1.invert_yaxis()
else:
    ax1.text(0.5, 0.5, 'No exogenous variables used', ha='center', va='center', transform=ax1.transAxes)
    ax1.set_title('Exogenous Variable Usage')

# ARIMA order distribution
orders_list = list(model_orders.keys())[:10]
counts_list = [model_orders[order] for order in orders_list]
ax2.bar(range(len(orders_list)), counts_list, color='lightblue')
ax2.set_xlabel('ARIMA Order')
ax2.set_ylabel('Number of Models')
ax2.set_title('Distribution of ARIMA Orders')
ax2.set_xticks(range(len(orders_list)))
ax2.set_xticklabels(orders_list, rotation=45)

# Model performance distribution
individual_maes = []
for name, fitted_values in fitted_values_dict.items():
    if name in ts_data_dict:
        try:
            ts_data = ts_data_dict[name]
            y_actual = ts_data['Weekly_Sales'].dropna()
            common_index = y_actual.index.intersection(fitted_values.index)

            if len(common_index) > 0:
                y_aligned = y_actual.loc[common_index]
                fitted_aligned = fitted_values.loc[common_index]
                mae = mean_absolute_error(y_aligned, fitted_aligned)
                individual_maes.append(mae)
        except:
            continue

if individual_maes:
    ax3.hist(individual_maes, bins=20, alpha=0.7, color='lightgreen')
    ax3.set_xlabel('MAE')
    ax3.set_ylabel('Frequency')
    ax3.set_title('Distribution of Individual Model MAE')
else:
    ax3.text(0.5, 0.5, 'No performance data available', ha='center', va='center', transform=ax3.transAxes)

# Seasonal order distribution
if seasonal_orders:
    seasonal_list = list(seasonal_orders.keys())[:8]
    seasonal_counts = [seasonal_orders[order] for order in seasonal_list]
    ax4.bar(range(len(seasonal_list)), seasonal_counts, color='coral')
    ax4.set_xlabel('Seasonal Order')
    ax4.set_ylabel('Number of Models')
    ax4.set_title('Distribution of Seasonal Orders')
    ax4.set_xticks(range(len(seasonal_list)))
    ax4.set_xticklabels(seasonal_list, rotation=45)
else:
    ax4.text(0.5, 0.5, 'No seasonal orders found', ha='center', va='center', transform=ax4.transAxes)
    ax4.set_title('Seasonal Order Distribution')

plt.tight_layout()
plt.savefig('final_arima_analysis.png', dpi=300, bbox_inches='tight')
wandb.log({"final_feature_importance": wandb.Image('final_arima_analysis.png')})
plt.show()

# Log analysis results
if not importance_df.empty:
    wandb.log({
        'top_10_features': importance_df.head(10)['Feature'].tolist(),
        'top_features_table': wandb.Table(dataframe=importance_df.head(10)),
        'total_features': len(feature_names)
    })

wandb.log({
    'total_models_fitted': total_models,
    'unique_arima_orders': len(model_orders),
    'unique_seasonal_orders': len(seasonal_orders),
    'avg_individual_mae': np.mean(individual_maes) if individual_maes else 0,
    'std_individual_mae': np.std(individual_maes) if individual_maes else 0
})

# MODEL PERFORMANCE VISUALIZATION

In [ ]:
# Model performance visualization
print("\n=== MODEL PERFORMANCE VISUALIZATION ===")

# Create performance plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Actual vs Predicted (sample)
sample_size = min(5000, len(y_final_aligned))
sample_indices = np.random.choice(len(y_final_aligned), size=sample_size, replace=False)
sample_actual = y_final_aligned[sample_indices]
sample_pred = y_pred_final[sample_indices]

axes[0, 0].scatter(sample_actual, sample_pred, alpha=0.5, s=1)
axes[0, 0].plot([y_final_aligned.min(), y_final_aligned.max()], [y_final_aligned.min(), y_final_aligned.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Sales')
axes[0, 0].set_ylabel('Predicted Sales')
axes[0, 0].set_title(f'Actual vs Predicted Sales (n={sample_size})')

# 2. Residuals
residuals = y_final_aligned - y_pred_final
axes[0, 1].scatter(y_pred_final[sample_indices], residuals[sample_indices], alpha=0.5, s=1)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted Sales')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residual Plot')

# 3. Residual distribution
axes[1, 0].hist(residuals, bins=100, alpha=0.7, color='lightgreen')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of Residuals')

# 4. Performance by individual models
if individual_maes:
    model_names = [f"Model {i+1}" for i in range(len(individual_maes))]
    sample_models = min(20, len(individual_maes))

    axes[1, 1].bar(range(sample_models), sorted(individual_maes)[:sample_models], color='coral')
    axes[1, 1].set_xlabel('Individual Models (sorted by MAE)')
    axes[1, 1].set_ylabel('MAE')
    axes[1, 1].set_title(f'MAE by Individual ARIMA Models (Best {sample_models})')
    axes[1, 1].tick_params(axis='x', rotation=45)
else:
    axes[1, 1].text(0.5, 0.5, 'No individual model data', ha='center', va='center', transform=axes[1, 1].transAxes)

plt.tight_layout()
plt.savefig('final_model_performance.png', dpi=300, bbox_inches='tight')
wandb.log({"final_model_performance": wandb.Image('final_model_performance.png')})
plt.show()

# Residual statistics
print(f"\nResidual Analysis:")
print(f"Residual Mean: {residuals.mean():.2f}")
print(f"Residual Std: {residuals.std():.2f}")
print(f"Residual Skewness: {residuals.skew():.3f}")
print(f"Residual Kurtosis: {residuals.kurtosis():.3f}")

wandb.log({
    'residual_mean': residuals.mean(),
    'residual_std': residuals.std(),
    'residual_skewness': residuals.skew(),
    'residual_kurtosis': residuals.kurtosis()
})

# SAVING FINAL MODEL

In [ ]:
# Save final model
print("\n=== SAVING FINAL MODEL ===")

# Fix MAPE calculation (handle zero values)
def safe_mape(y_true, y_pred):
    """Calculate MAPE while handling zero values"""
    mask = y_true != 0
    if mask.sum() == 0:
        return float('inf')
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

train_mape_fixed = safe_mape(y_final_aligned, y_pred_final)
print(f"Corrected Training MAPE: {train_mape_fixed:.2f}%")

# Save pipeline with cloudpickle (handles custom classes better)
try:
    import cloudpickle
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'cloudpickle'])
    import cloudpickle

# Save both preprocessor and model
pipeline_filename = f"arima_pipeline_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"

# Create save object with both components
pipeline_save_object = {
    'preprocessor': final_preprocessor,
    'model': final_model,
    'ts_data_dict': ts_data_dict,
    'performance_metrics': {
        'train_mae': train_mae,
        'train_rmse': train_rmse,
        'train_mape': train_mape_fixed,
        'train_r2': train_r2
    },
    'metadata': {
        'successful_fits': successful_fits,
        'failed_fits': failed_fits,
        'total_models': total_models,
        'exog_variables': feature_names,
        'created_date': datetime.now().isoformat()
    }
}

# Save with cloudpickle
with open(pipeline_filename, 'wb') as f:
    cloudpickle.dump(pipeline_save_object, f)

print(f"Pipeline saved as: {pipeline_filename}")

# Try to upload to wandb with error handling
try:
    # Create model artifact with fixed metadata
    model_artifact = wandb.Artifact(
        name="ARIMA_pipeline",
        type="model",
        description="Final ARIMA pipeline for Walmart sales forecasting",
        metadata={
            "train_mae": float(train_mae),
            "train_rmse": float(train_rmse),
            "train_mape": float(train_mape_fixed) if not np.isinf(train_mape_fixed) else 0.0,
            "train_r2": float(train_r2),
            "features_count": len(feature_names),
            "training_samples": len(y_final_aligned),
            "model_type": "ARIMA",
            "individual_models": total_models,
            "successful_fits": successful_fits
        }
    )

    # Add model file to artifact
    model_artifact.add_file(pipeline_filename)

    # Log artifact
    wandb.log_artifact(model_artifact)
    print("✓ Model artifact logged to wandb successfully!")

except Exception as e:
    print(f"⚠️ Error uploading to wandb: {e}")
    print("Model saved locally - you can manually upload later")

    # Log just the metrics without artifact
    wandb.log({
        'final_train_mae': train_mae,
        'final_train_rmse': train_rmse,
        'final_train_mape': train_mape_fixed if not np.isinf(train_mape_fixed) else 0.0,
        'final_train_r2': train_r2,
        'model_saved_locally': pipeline_filename
    })

# Final summary with corrected MAPE
print(f"\n" + "="*60)
print("FINAL MODEL SUMMARY")
print("="*60)
print(f"Model Type: ARIMA Ensemble")
print(f"Training Samples: {len(y_final_aligned):,}")
print(f"Individual Models: {total_models}")
print(f"Successful Fits: {successful_fits}")
print(f"Features: {len(feature_names)}")
print(f"Training MAE: {train_mae:.2f}")
print(f"Training RMSE: {train_rmse:.2f}")
print(f"Training MAPE: {train_mape_fixed:.2f}%")
print(f"Training R²: {train_r2:.4f}")
print(f"Pipeline saved as: {pipeline_filename}")
print("="*60)

wandb.finish()